In [1]:
!pip install langgraph langchain_core langchain_groq langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 8.0 MB/s eta 0:00:00


In [2]:
!pip install duckduckgo-search gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00


In [3]:
"""
Tourism Domain Ontology

This ontology is built from scratch for the tourism domain, incorporating:
- Tourist (profile, preferences)
- Destination (location, attractions, accommodation)
- Activities (categories, durations, costs)
- Environmental factors (weather, season)
- Trip logistics (duration, budget, transportation)

The ontology is represented as a structured knowledge base with relationships
between these entities defined implicitly through the recommendation logic.
"""

'\nTourism Domain Ontology\n\nThis ontology is built from scratch for the tourism domain, incorporating:\n- Tourist (profile, preferences)\n- Destination (location, attractions, accommodation)\n- Activities (categories, durations, costs)\n- Environmental factors (weather, season)\n- Trip logistics (duration, budget, transportation)\n\nThe ontology is represented as a structured knowledge base with relationships\nbetween these entities defined implicitly through the recommendation logic.\n'

In [4]:
from google.colab import userdata
import os
os.environ['GROQ_API_KEY'] = userdata.get('groq_api_key')

In [6]:
import os
from typing import TypedDict, Annotated, List, Dict, Any
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import requests
from datetime import datetime
import json

"""
Tourism Domain Ontology

This ontology is built from scratch for the tourism domain, incorporating:
- Tourist (profile, preferences)
- Destination (location, attractions, accommodation)
- Activities (categories, durations, costs)
- Environmental factors (weather, season)
- Trip logistics (duration, budget, transportation)

The ontology is represented as a structured knowledge base with relationships
between these entities defined implicitly through the recommendation logic.

The system follows a simple yet effective workflow:
1. Collect tourist profile information
2. Get destination details and preferences
3. Fetch weather and accommodation information
4. Generate a personalized itinerary
5. Collect simple feedback (awesome/good/ok)
6. Refine itinerary if needed or finalize

This creates a complete tourism recommendation system that's
easy to use and provides personalized recommendations.
"""

# Define the State Type for our Tourism Recommendation Agent
class TourismRecommenderState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
    tourist_profile: Dict[str, Any]  # Age group, travel style, etc.
    destination: str  # City or location
    duration: int  # Number of days
    budget: str  # Budget category
    interests: List[str]  # Tourist interests
    weather_info: Dict[str, Any]  # Weather information
    accommodation_preference: str  # Hotel preference
    hotel_info: Dict[str, Any]  # Hotel information
    itinerary: str  # Generated itinerary
    feedback: str  # User feedback on itinerary
    feedback_rating: str  # Simple rating (awesome/good/ok)
    iteration: int  # Number of iterations for itinerary generation

# Set up the LLM (make sure to set your API key in the environment)
# You should set your GROQ_API_KEY as an environment variable
llm = ChatGroq(
    temperature=0.5,
    model_name="llama-3.3-70b-versatile"
)

# DuckDuckGo Search Function
def duckduckgo_search(query, num_results=5):
    """Perform a DuckDuckGo search using their API."""
    url = "https://api.duckduckgo.com/"
    params = {
        'q': query,
        'format': 'json',
        'pretty': '1',
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            # Parse the results from DuckDuckGo
            results = response.json()
            return results
        else:
            return {"error": f"Search failed with status code {response.status_code}"}
    except Exception as e:
        return {"error": f"Search failed: {str(e)}"}

# Simulate weather API with DuckDuckGo search
def get_weather_info(destination):
    """Get weather information for a destination using DuckDuckGo."""
    search_query = f"current weather in {destination}"
    results = duckduckgo_search(search_query)

    # For simulation, we'll parse information from the search results
    # In a real app, you would use a dedicated weather API
    weather_info = {
        "destination": destination,
        "current_temp": "25°C",  # Simulated value
        "conditions": "Partly Cloudy",  # Simulated value
        "forecast": "Expected to remain pleasant throughout the week"  # Simulated value
    }

    return weather_info

# Get hotel recommendations
def get_hotel_recommendations(destination, preference):
    """Get hotel recommendations based on destination and preferences."""
    search_query = f"best {preference} hotels in {destination}"
    results = duckduckgo_search(search_query)

    # For simulation, we'll return some fixed data
    # In a real app, you would parse the search results more carefully
    hotels = {
        "budget": {
            "name": f"Budget Stay {destination}",
            "price_range": "$50-100 per night",
            "rating": "3.5 stars",
            "amenities": ["Free WiFi", "Breakfast included"]
        },
        "mid-range": {
            "name": f"Comfort Inn {destination}",
            "price_range": "$100-200 per night",
            "rating": "4 stars",
            "amenities": ["Pool", "Gym", "Restaurant", "Free WiFi"]
        },
        "luxury": {
            "name": f"Grand Hotel {destination}",
            "price_range": "$300+ per night",
            "rating": "5 stars",
            "amenities": ["Spa", "Multiple Restaurants", "Premium Services", "Ocean View"]
        }
    }

    # Return the appropriate hotel category or a default if not found
    return hotels.get(preference.lower(), hotels["mid-range"])

# Define Prompt Templates
profile_prompt = """Based on the following information, generate a tourist profile summary:
- Age group: {age_group}
- Travel style: {travel_style}
- Previous travel experience: {experience}

Create a brief profile that will help tailor tourism recommendations."""

itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a tourism recommendation expert with deep knowledge of destinations worldwide.

    Create a personalized travel itinerary based on the following information:

    TOURIST PROFILE:
    {tourist_profile}

    DESTINATION: {destination}
    DURATION: {duration} days
    BUDGET: {budget}
    INTERESTS: {interests}

    WEATHER INFO:
    {weather_info}

    ACCOMMODATION:
    {hotel_info}

    Create a detailed day-by-day itinerary that includes:
    1. Morning, afternoon, and evening activities
    2. Recommended local food experiences
    3. Transportation suggestions between attractions
    4. Estimated costs for activities
    5. Tips based on current weather conditions

    Format the itinerary in a clear, readable structure with days clearly marked.
    """),
    ("user", "Please generate my personalized travel itinerary.")
])

feedback_analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI assistant that analyzes feedback on travel itineraries.

    Analyze the following feedback on a travel itinerary and determine if it's generally positive or negative.
    Return a simple analysis with:
    1. The overall sentiment (positive/negative/mixed)
    2. Key points from the feedback
    3. Specific areas for improvement if any

    FEEDBACK: {feedback}

    Format your response as a JSON object with the following structure:
    {
        "sentiment": "positive/negative/mixed",
        "key_points": ["point1", "point2", ...],
        "areas_for_improvement": ["area1", "area2", ...]
    }
    """),
    ("user", "Please analyze this travel itinerary feedback.")
])

# Node Functions for the Graph

def collect_tourist_profile(state: TourismRecommenderState) -> TourismRecommenderState:
    """Collect basic tourist profile information."""
    print("\n===== TOURIST PROFILE =====")

    print("\nWhat is your age group?")
    print("1. 18-25")
    print("2. 26-40")
    print("3. 41-60")
    print("4. 60+")
    age_choice = input("Enter the number of your choice: ")
    age_map = {"1": "18-25", "2": "26-40", "3": "41-60", "4": "60+"}
    age_group = age_map.get(age_choice, "26-40")

    print("\nWhat is your travel style?")
    print("1. Adventure seeker")
    print("2. Cultural explorer")
    print("3. Relaxation oriented")
    print("4. Luxury traveler")
    print("5. Budget backpacker")
    style_choice = input("Enter the number of your choice: ")
    style_map = {
        "1": "Adventure seeker",
        "2": "Cultural explorer",
        "3": "Relaxation oriented",
        "4": "Luxury traveler",
        "5": "Budget backpacker"
    }
    travel_style = style_map.get(style_choice, "Cultural explorer")

    print("\nHow would you describe your travel experience?")
    print("1. Novice (first time traveling)")
    print("2. Occasional traveler")
    print("3. Experienced traveler")
    print("4. Expert globetrotter")
    exp_choice = input("Enter the number of your choice: ")
    exp_map = {
        "1": "Novice",
        "2": "Occasional traveler",
        "3": "Experienced traveler",
        "4": "Expert globetrotter"
    }
    experience = exp_map.get(exp_choice, "Occasional traveler")

    # Generate a profile summary using LLM
    profile_summary = llm.invoke(
        profile_prompt.format(
            age_group=age_group,
            travel_style=travel_style,
            experience=experience
        )
    ).content

    tourist_profile = {
        "age_group": age_group,
        "travel_style": travel_style,
        "experience": experience,
        "summary": profile_summary
    }

    return {
        **state,
        "tourist_profile": tourist_profile,
        "messages": state["messages"] + [AIMessage(content=f"Tourist profile created: {profile_summary}")],
        "iteration": 1  # Initialize iteration count
    }

def collect_destination_info(state: TourismRecommenderState) -> TourismRecommenderState:
    """Collect destination and trip details."""
    print("\n===== DESTINATION & TRIP DETAILS =====")

    destination = input("\nEnter the destination city you want to visit: ")

    print("\nHow many days will you be staying?")
    duration_str = input("Enter number of days: ")
    try:
        duration = int(duration_str)
    except ValueError:
        duration = 3  # Default to 3 days if invalid input
        print(f"Using default duration of {duration} days")

    print("\nWhat is your budget level for this trip?")
    print("1. Budget (economy)")
    print("2. Mid-range")
    print("3. Luxury")
    budget_choice = input("Enter the number of your choice: ")
    budget_map = {"1": "budget", "2": "mid-range", "3": "luxury"}
    budget = budget_map.get(budget_choice, "mid-range")

    print("\nWhat are your interests for this trip? (comma-separated)")
    print("Examples: history, food, nature, shopping, museums, adventure, relaxation, nightlife")
    interests_input = input("Enter your interests: ")
    interests = [interest.strip() for interest in interests_input.split(",")]

    return {
        **state,
        "destination": destination,
        "duration": duration,
        "budget": budget,
        "interests": interests,
        "messages": state["messages"] + [AIMessage(content=f"Destination details collected for {destination}, {duration} days, {budget} budget")],
    }

def get_accommodation_info(state: TourismRecommenderState) -> TourismRecommenderState:
    """Collect accommodation preferences and get recommendations."""
    print("\n===== ACCOMMODATION PREFERENCES =====")

    print("\nWhat type of accommodation are you looking for?")
    print("1. Budget (hostel, guesthouse)")
    print("2. Mid-range (3-star hotel)")
    print("3. Luxury (4-5 star hotel)")
    acc_choice = input("Enter the number of your choice: ")
    acc_map = {"1": "budget", "2": "mid-range", "3": "luxury"}
    accommodation_preference = acc_map.get(acc_choice, "mid-range")

    # Get hotel recommendations using DuckDuckGo
    print(f"\nSearching for {accommodation_preference} hotels in {state['destination']}...")
    hotel_info = get_hotel_recommendations(state['destination'], accommodation_preference)

    # Display hotel information
    print(f"\nFound: {hotel_info['name']}")
    print(f"Price range: {hotel_info['price_range']}")
    print(f"Rating: {hotel_info['rating']}")
    print(f"Amenities: {', '.join(hotel_info['amenities'])}")

    return {
        **state,
        "accommodation_preference": accommodation_preference,
        "hotel_info": hotel_info,
        "messages": state["messages"] + [AIMessage(content=f"Recommended accommodation: {hotel_info['name']}")],
    }

def get_weather_data(state: TourismRecommenderState) -> TourismRecommenderState:
    """Get weather information for the destination."""
    print("\n===== WEATHER INFORMATION =====")

    print(f"\nChecking current weather in {state['destination']}...")
    weather_info = get_weather_info(state['destination'])

    print(f"Current temperature: {weather_info['current_temp']}")
    print(f"Conditions: {weather_info['conditions']}")
    print(f"Forecast: {weather_info['forecast']}")

    return {
        **state,
        "weather_info": weather_info,
        "messages": state["messages"] + [AIMessage(content=f"Weather information retrieved for {state['destination']}")],
    }

def generate_itinerary(state: TourismRecommenderState) -> TourismRecommenderState:
    """Generate a personalized itinerary using the LLM."""
    print("\n===== GENERATING PERSONALIZED ITINERARY =====")
    print(f"Creating itinerary for {state['destination']} ({state['duration']} days)...")

    # Format information for the prompt
    weather_info_str = f"Temperature: {state['weather_info']['current_temp']}, Conditions: {state['weather_info']['conditions']}, Forecast: {state['weather_info']['forecast']}"

    hotel_info_str = f"Accommodation: {state['hotel_info']['name']}, {state['hotel_info']['price_range']}, {state['hotel_info']['rating']}"

    # Generate the itinerary
    response = llm.invoke(
        itinerary_prompt.format_messages(
            tourist_profile=state['tourist_profile']['summary'],
            destination=state['destination'],
            duration=state['duration'],
            budget=state['budget'],
            interests=', '.join(state['interests']),
            weather_info=weather_info_str,
            hotel_info=hotel_info_str
        )
    )

    itinerary = response.content

    print("\n===== YOUR PERSONALIZED ITINERARY =====\n")
    print(itinerary)

    return {
        **state,
        "itinerary": itinerary,
        "messages": state["messages"] + [AIMessage(content=itinerary)],
    }

def collect_feedback(state: TourismRecommenderState) -> TourismRecommenderState:
    """Collect user feedback on the generated itinerary."""
    print("\n===== FEEDBACK =====")
    print("What do you think about this itinerary?")
    print("Type 'awesome' if you love it, 'good' if it's fine, or 'ok' if you want changes:")

    feedback = input("Your feedback (awesome/good/ok): ").strip().lower()

    # Add any additional comments
    print("Any specific comments or suggestions for improvement?")
    additional_comments = input("Additional comments (optional): ")

    # Combine the rating with additional comments
    full_feedback = f"Rating: {feedback}. Additional comments: {additional_comments}"

    return {
        **state,
        "feedback": full_feedback,
        "feedback_rating": feedback,  # Store just the rating for easier decision making
        "messages": state["messages"] + [HumanMessage(content=full_feedback)],
    }

def analyze_feedback(state: TourismRecommenderState) -> dict:
    """Analyze user feedback and determine next steps based on simple keywords."""
    print("\n===== ANALYZING FEEDBACK =====")

    # Get the feedback rating
    feedback_rating = state.get("feedback_rating", "").strip().lower()

    # Simple keyword-based decision making
    if feedback_rating in ["awesome", "good"]:
        print("Great! You seem happy with the itinerary.")
        return {"next": "end"}
    else:  # "ok" or any other input
        print("Let's refine the itinerary based on your feedback.")
        return {"next": "refine_itinerary"}

def refine_itinerary(state: TourismRecommenderState) -> TourismRecommenderState:
    """Refine the itinerary based on user feedback."""
    print("\n===== REFINING ITINERARY =====")
    print("Based on your feedback, we'll create an improved itinerary.")

    # Increment the iteration counter
    iteration = state["iteration"] + 1

    # Get additional refinement requests
    print("\nPlease tell us what specific aspects you'd like to change:")
    refinement_requests = input("What would you like to change? ")

    # Create a refinement prompt that includes the original itinerary and feedback
    refinement_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a tourism recommendation expert with deep knowledge of destinations worldwide.

        REFINE the following travel itinerary based on user feedback:

        ORIGINAL ITINERARY:
        {original_itinerary}

        USER FEEDBACK:
        {feedback}

        SPECIFIC CHANGES REQUESTED:
        {refinement_requests}

        TOURIST PROFILE:
        {tourist_profile}

        DESTINATION: {destination}
        DURATION: {duration} days
        BUDGET: {budget}
        INTERESTS: {interests}

        Create an IMPROVED day-by-day itinerary that addresses the specific feedback while maintaining the positive aspects of the original plan.

        Format the itinerary in a clear, readable structure with days clearly marked.
        """),
        ("user", "Please generate my refined travel itinerary.")
    ])

    # Generate the refined itinerary
    response = llm.invoke(
        refinement_prompt.format_messages(
            original_itinerary=state["itinerary"],
            feedback=state["feedback"],
            refinement_requests=refinement_requests,
            tourist_profile=state['tourist_profile']['summary'],
            destination=state['destination'],
            duration=state['duration'],
            budget=state['budget'],
            interests=', '.join(state['interests'])
        )
    )

    refined_itinerary = response.content

    print("\n===== YOUR REFINED ITINERARY =====\n")
    print(refined_itinerary)

    return {
        **state,
        "itinerary": refined_itinerary,
        "messages": state["messages"] + [AIMessage(content=refined_itinerary)],
        "iteration": iteration
    }

def finalize_itinerary(state: TourismRecommenderState) -> TourismRecommenderState:
    """Finalize the itinerary and provide a summary."""
    print("\n===== FINAL ITINERARY =====")
    print("Thank you for using our Tourism Recommendation System!")
    print(f"Your {state['duration']}-day itinerary for {state['destination']} is ready.")
    print("The itinerary is: ")
    print(state["itinerary"])

    # Create a summary of the itinerary
    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", """Summarize the key highlights of this travel itinerary in 3-5 bullet points:

        {itinerary}

        Format as concise bullet points starting with emoji indicators.
        """),
        ("user", "Please summarize this travel itinerary.")
    ])

    summary_response = llm.invoke(
        summary_prompt.format_messages(
            itinerary=state["itinerary"]
        )
    )

    print("\n===== ITINERARY HIGHLIGHTS =====")
    print(summary_response.content)
    print("\nWe hope you enjoy your trip!")

    return {
        **state,
        "messages": state["messages"] + [AIMessage(content=summary_response.content)],
    }

# Create and compile the graph
def create_tourism_recommendation_system():
    """Create the tourism recommendation system graph."""
    # Create the workflow graph
    workflow = StateGraph(TourismRecommenderState)

    # Add nodes
    workflow.add_node("collect_tourist_profile", collect_tourist_profile)
    workflow.add_node("collect_destination_info", collect_destination_info)
    workflow.add_node("get_weather_data", get_weather_data)
    workflow.add_node("get_accommodation_info", get_accommodation_info)
    workflow.add_node("generate_itinerary", generate_itinerary)
    workflow.add_node("collect_feedback", collect_feedback)
    workflow.add_node("analyze_feedback", analyze_feedback)
    workflow.add_node("refine_itinerary", refine_itinerary)
    workflow.add_node("finalize_itinerary", finalize_itinerary)

    # Set entry point
    workflow.set_entry_point("collect_tourist_profile")

    # Define edges
    workflow.add_edge("collect_tourist_profile", "collect_destination_info")
    workflow.add_edge("collect_destination_info", "get_weather_data")
    workflow.add_edge("get_weather_data", "get_accommodation_info")
    workflow.add_edge("get_accommodation_info", "generate_itinerary")
    workflow.add_edge("generate_itinerary", "collect_feedback")
    workflow.add_conditional_edges(
        "analyze_feedback",
        lambda x: x["next"],
        {
            "refine_itinerary": "refine_itinerary",
            "end": "finalize_itinerary"
        }
    )
    workflow.add_edge("collect_feedback", "analyze_feedback")
    workflow.add_edge("refine_itinerary", "collect_feedback")
    workflow.add_edge("finalize_itinerary", END)

    # Compile the graph
    return workflow.compile()

# Function to run the system
def run_tourism_recommendation_system():
    """Run the Tourism Recommendation System."""
    print("=" * 50)
    print("WELCOME TO THE TOURISM RECOMMENDATION SYSTEM")
    print("=" * 50)
    print("\nThis system will help you plan your perfect trip based on your preferences.")

    # Initialize the state
    state = {
        "messages": [],
        "tourist_profile": {},
        "destination": "",
        "duration": 0,
        "budget": "",
        "interests": [],
        "weather_info": {},
        "accommodation_preference": "",
        "hotel_info": {},
        "itinerary": "",
        "feedback": "",
        "feedback_rating": "",
        "iteration": 0
    }

    # Create and run the graph
    app = create_tourism_recommendation_system()

    # Execute the graph
    for output in app.stream(state):
        # We can observe each state transition here if needed
        pass

# Main execution
if __name__ == "__main__":
    run_tourism_recommendation_system()

WELCOME TO THE TOURISM RECOMMENDATION SYSTEM

This system will help you plan your perfect trip based on your preferences.

===== TOURIST PROFILE =====

What is your age group?
1. 18-25
2. 26-40
3. 41-60
4. 60+
Enter the number of your choice: 1

What is your travel style?
1. Adventure seeker
2. Cultural explorer
3. Relaxation oriented
4. Luxury traveler
5. Budget backpacker
Enter the number of your choice: 2

How would you describe your travel experience?
1. Novice (first time traveling)
2. Occasional traveler
3. Experienced traveler
4. Expert globetrotter
Enter the number of your choice: 2

===== DESTINATION & TRIP DETAILS =====

Enter the destination city you want to visit: Tirupati 

How many days will you be staying?
Enter number of days: 3

What is your budget level for this trip?
1. Budget (economy)
2. Mid-range
3. Luxury
Enter the number of your choice: 1

What are your interests for this trip? (comma-separated)
Examples: history, food, nature, shopping, museums, adventure, rela